# Laboratoire 3 : Machines à vecteur de support et réseaux neuronaux
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | Alexandre Bleau — BLEA14058906 / David Létinaud  — LETD05129708 / Thomas Lioret   — LIOT20069605|
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2019                                            |
| Groupe                | 1                                                       |
| Numéro du laboratoire | 3                                                       |
| Professeur            | Prof. LOMBAERT                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 18/11/2019                                              |

## Introduction

Au cours de ce troisième laboratoire nous allons étudier deux nouveaux algorithme de classification pour résoudre le problème de classification des galaxies : les réseaux neuronaux et les machines à vecteurs de support (SVM). 
Dans un premier temps, nous allons concevoir un modèle de réseaux neuronaux basé sur le Multi-Layer Perceptron. Nous entrainerons ce modèle afin qu'il puisse classer les galaxies en "smooth" ou "spiral" en utilisant l'ensemble des primitives. Nous utiliserons le module keras de Google tensorflow
Le deuxième modèle d'apprentissage s'appuie sur un modèle d'optimisation convexe dans le cas du SVM. Dans ce cas, nous n'utiliserons qu'une partie des primitives proposées et seront couplées à nos primitives développées lors du premier laboratoire.
Nous étudierons également l'influence des hyperparamètres de ces deux méthodes afin de proposer le modèle le plus optimal dans le cas de notre problème de classification de galaxies.

## Question 1
(grille de correction : Les approches de validations sont présentées et utilisées correctement.)

Dans le cas des réseaux neuronaux, l'utilisation d'une méthode de validation croisée prendrait beaucoup de temps. En effet, il faudrait répéter plusieurs tests avec un nombre "d'epochs" conséquent : ceci serait très chronophage. Nous avons donc décider d'utiliser la méthode de validation hold-out avec 70% de donées d'entrainement et 30% de test.

Concernant la méthode SVM, nous utilisons la méthode ......... (k-fold????)


## Question 2
(La méthode de normalisation des données est décrite et correcte.)

Nous normalisons nos données grâce à la méthode "normalize" de la librairie preprocessing. La normalisation se fait par rapport à la valeur maximale. Nous avons décider de normaliser par rapport aux primitives (axis = 0). Par ailleurs, nous avons remarqué avec nos premier tests que les performances étaient meilleures si l'on normalisait par rapport à la valeur maximale (norm = 'max').

## Question 3
(Le modèle MLP est décrit (structure, nombre de couches). La fonction de coût choisie est mentionnée ainsi que les raisons pour lesquelles elle a été choisie.)

La principale contrainte du modèle MLP est de trouver un compromis entre le temps d'apprentissage et l'accuracy. La première architecture proposée par l'énoncé est de trois couches : 100, 100, 2 (nombre de perceptrons). Nous avons étudié différents cas avec 60 epochs :  
1 - l'influence du nombre de perceptrons avec un nombre de couche fixé.  
2 - l'influence du nombre de couches avec le même nombre de perceptrons par couche.

Voici donc nos résultats avec différentes structures de réseaux de neurones :

1 - a:
(Graphes nombre de perceptrons)

Nous remarquons qu'avec trois couches, les performances d'accuracy et de f1_score sont meilleurs dans le cas de [500,500,500] (dépassant les 95%). On remarque également que la perte ("loss") est, en quelque sorte, inversement proportionnelle aux f1_score et accuracy dans ce cas. C'est à dire que pour un f1_score et une accuracy plus faible (nombre de perceptrons inférieur) la valeur de perte sera plus importante que les architectures avec plus de perceptrons.

(graphe time nb perceptrons)

En revanche, ajouter un nombre de perceptrons par couche important influence grandement le temps d'entrainement et de prediction.


1 - b:
(graphe nombre de couches)

Dans notre problème de classification de galaxies, on note que le les valeurs des accuracy et des f1_scores tendent vers les mêmes performances sur les jeux de données de tests qu'il y ait une, trois ou six couches (nb de perceptrons constant par couche). On remarque également que la valeur de perte est moins importante dans le cas où il y le moins de couche. 

(graphe time nb couche)
Ici, nous remarquons que : plus le nombre de couche est important plus le temps d'entrainement et de prediction sont élevés.


## Question 4
(L’analyse est claire et l’équipe démontre une compréhension du phénomène de sur-apprentissage. Il le phénomène est correctement décrit et montré dans le graphique dans la mesure du possible.)

(graphe nb d'epoch/perf)

EXPLICATION A REVOIR CAR JE NE VOIS PAS DE SURAPPRENTISSAGE : 
Nous remarquons que les performances (accuracy et f1_score) tendent vers les mêmes taux qu'il y ait 30, 60 ou 120 itérations. 

(graphe time - itérations)

Notons ici que le temps d'apprentissage et de prédiction croissent avec le nombre d'itération.

## Question 5
(1 - La démarche de comparaison des hyperparamètres est sérieuse. Les résultats sont présentés de façon correcte et concise dans un tableau et un graphique.
2 - Les explications montrant les différences sont claires, concises et plausibles.
3 - La configuration matérielle sur laquelle les expérimentations est présenteainsi que le temps d’exécution requis pour compléter les expérimentations)

1&2 : (alterner graphe et explication (cf q3) ou mettre tous les graphes (en q3) puis explication ici en q5.

3 : regarder les specs du PC.

## Question 6
(1 - La méthode est décrite et un lien avec l’implémentation est fait afin d’expliquer comment l’équipe a trouvé le meilleur modèle SVM. 
2- Les résultats sont présentés de façon correcte et concise dans un TABLEAU ET un GRAPHIQUE. 
3 - La configuration matérielle sur laquelle les expérimentations est présente ainsi que le temps d’exécution requis pour compléter les expérimentations.)

1 - explication implémentation SVM

## Question 7
(L’impact de la taille de l’ensemble d’apprentissage sur les performances est présent et décrit convenablement.)

Dans le cas du MLP, on remarque que pour 1600 images, le temps d'apprentissage se situe autour de 5 secondes. Pour 16000, le temps d'apprentissage est d'environ 32 secondes. 

## Question 8
(Un classificateur est recommandé en se basant sur l’expérimentation précédemment effectuée.)

SVM ou MLP en fonction des résultats (perf + time).

## Question 9
(Des pistes d’amélioration sont proposées.)

Bonne question : investiguer davantages les études d'hyperparamètres pour trouver une solution optimale ?

## Conlusion

(1 - Un court résumé du problème est présent.
2 - Un rappel des résultats est présent.
3 - Des pistes pour de possibles améliorations sont présentes.)

Ce troisième laboratoire nous a permis de comprendre davantage le fonctionnment de deux nouvelles méthodes de classification : SVM et MLP. Dans les deux cas, les temps d'apprentissage sont bien plus long que les premiers laboratoires. 
Avantages et incovénients entre les deux méthodes:
MLP : nb de sorties non limitée != svm => MLP plus évolutif si plus de deux catégories. 

Les résultats sont meilleurs dans le cas de ..... (MLP,SVM ?).

